In [1]:
import json
import requests
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import urllib.request
import io
import numpy as np
import re
import pandas as pd

In [2]:
# chosen_movies = { 102720:4.5,
#                103335:5.0,
#                104076:4.5,
#                106696:4.5,
#                157340:5.0 }
# rec_url   = 'http://127.0.0.1:5000/api/recommend'

# response = requests.post(rec_url, json=chosen_movies)
# recommendations = response.json()
# recommendations

In [3]:
# Configs
items_per_row       = 6
rows                = 2
items_per_column    = 3

thumbnail_dimensions = (210, 200)
images_dim           = (500,500)
title_font           = 12
style                = 'Arial'
background_color     = "lightblue"
contrast_color       = "lavender"

default_rating = 0.0
rating_options   = ['-', 5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, 0.5]

zoom  = 'zoomed'
title = 'Movie Recommender'

# API details
api_key   = "dd42205e97e18a6c1a1be23521afb906"
base_url  = "https://api.themoviedb.org/3/movie/"
image_url = "https://image.tmdb.org/t/p/original"
rec_url   = 'http://127.0.0.1:5000/api/recommend'

# Global lists/dicts
new_user_ratings = {}

MODEL_PATH = 'api_calls/'

In [4]:
# The TK window objects
class MovieDisplay:
    def __init__(self, root, movie_id, api_key):
        self.id = movie_id
        link_id = movies_df[movies_df['movieId']==movie_id]['tmdbId'].iloc[0]

        response = requests.get(f"{base_url}{link_id}?api_key={api_key}")

        data     = json.loads(response.text)
        title    = data['title']

        # Get movie poster path
        poster_path = data["poster_path"]
        # Generate image URL
        image_path = f"{image_url}{poster_path}"

        # Download the image from its URL
        with urllib.request.urlopen(image_path) as url:
            image_data = url.read()

        # Load the image data into a PIL Image object
        image = Image.open(io.BytesIO(image_data))

        # Create a thumbnail
        image.thumbnail(thumbnail_dimensions)

        # Create a PhotoImage object from the resized PIL Image object
        self.photo = ImageTk.PhotoImage(image)

        # Create a canvas and display the image on it
        self.canvas = tk.Canvas(root, width=self.photo.width(), height=self.photo.height(), bg=background_color)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)
        self.canvas.image = self.photo

        self.title_label = tk.Label(root, text=title, font=(style, title_font), bg=background_color)
        self.title_label.configure(wraplength=self.photo.width())

        #Combobox for selecting the rating
        self.rating_combobox = ttk.Combobox(root, values=rating_options, font=(style, 12), state="readonly")
        self.rating_combobox.current(0)  # set the default value to the first rating option

        self.rating = default_rating

        self.rating_combobox.bind("<<ComboboxSelected>>", self.update_rating)

    def update_rating(self, event):
        new_rating = self.rating_combobox.get()
        if new_rating == '-':
            self.rating = default_rating
        else:
            self.rating = new_rating

In [5]:
# Converts the title to lowercase, remove special characters, and spaces
def clean_title(title):
    cleaned_title = re.sub(r'[^a-zA-Z0-9]', '', title.lower())
    return cleaned_title

In [6]:
def search_movies(term):
    print('Searching movies containing: '+ term)
    print('--------------------------------------')
    term = clean_title(term)
    result = movies_df[movies_df["cleaned_title"].str.contains(term)]
    
    result_ids = result["movieId"].tolist()
    result_ids = result_ids[0:items_per_row*rows]
    return result_ids

In [7]:
def get_recommended_movies(chosen_movies):
    
    response = requests.post(rec_url, json=chosen_movies)

    if response.status_code == 200:
        recommendations = response.json()
        
    else:
        print("Request failed with status code:", response.status_code)
        recommendations = []

    return recommendations

In [8]:
# Get some random movies
def get_movies(number_of_movies):
    
    movie_ids = movies_df['movieId'].tolist()
    
    random_movie_ids = np.random.choice(movie_ids, number_of_movies)
    random_movie_ids_list = random_movie_ids.tolist()
    
    return random_movie_ids_list

In [9]:
def display_movies(movie_list,mode):
    print('Displaying movies:')
    print(movie_list)
    
    # Clear all the widgets of the window
    for widget in root.winfo_children():
        widget.destroy()
    
    if mode == 'rate':    
        tk.Label(root, text='Βαθμολόγησε τις αγαπημένες σου (ή όχι) ταινίες!', bg=contrast_color).grid(row=0, column=items_per_row // 2 -1, columnspan=2)
    else:
        tk.Label(root, text='Προτεινόμενες ταινίες!', bg=contrast_color).grid(row=0, column=items_per_row // 2 - 1, columnspan=2)
    
    # MovieDisplay object for each movie
    movie_displays = []
    for i, movie in enumerate(movie_list):
        current_row = int(i / items_per_row)
        col         = i % items_per_row
        subrow      = int(current_row * items_per_column)

        display = MovieDisplay(root, movie, api_key)

        if mode == 'rate':
            display.rating_combobox.grid(column=col, row=subrow + 3, padx=10, pady=10)
            display.canvas.grid(column=col, row=subrow + 1, padx=10, pady=10)
            display.title_label.grid(column=col, row=subrow + 2, padx=10, pady=10)
        else:
            info_button = tk.Button(root, text="Περισσότερες πληροφορίες", font=(style, 12), bg=contrast_color, command=lambda movie=movie: show_info(movie))
            info_button.grid(column=col, row=subrow + 3, padx=(10, 100), pady=10)
            display.canvas.grid(column=col, row=subrow + 1, padx=(10, 100), pady=10)
            display.title_label.grid(column=col, row=subrow + 2, padx=(10, 100), pady=10)

        movie_displays.append(display)

    if mode == 'rate':
        submit_button = tk.Button(root, text="Πρότεινε μου ταινίες!", font=(style, 12), bg=contrast_color, command=lambda: button_clicked(movie_displays,'recommend'))
        submit_button.grid(column=items_per_row // 2 - 1, row=current_row + 7, padx=10, pady=10, columnspan=2)
        
        more_button = tk.Button(root, text="Τυχαίες ταινίες!", font=(style, 12), bg=contrast_color, command=lambda: button_clicked(movie_displays))
        more_button.grid(column=items_per_row // 2 - 1, row=current_row + 6, padx=10, pady=10, columnspan=2)
                
        search_entry = tk.Entry(root, font=(style, 12))
        search_entry.grid(column=items_per_row // 2 + 2, row=current_row + 6, padx=10, pady=10)
        
        search_button = tk.Button(root, text="Αναζήτηση ταινιών!", font=(style, 12), bg=contrast_color, command=lambda: button_clicked(movie_displays,'search',search_entry.get()))
        search_button.grid(column=items_per_row // 2 + 1, row=current_row + 6, padx=10, pady=10)
    else:
        retry_button = tk.Button(root, text="Νέα δοκιμή!", font=(style, 12), bg=contrast_color, command=retry)
        retry_button.grid(column=items_per_row // 2 - 1, row=current_row + 7, padx=10, pady=10)
        
        close_button = tk.Button(root, text="Κλείσιμο εφαρμογής!", font=(style, 12), bg=contrast_color, command=close)
        close_button.grid(column=items_per_row // 2, row=current_row + 7, padx=10, pady=10)

In [10]:
def retry():
    print('--------------------------------------')
    print('RESET')
    print('--------------------------------------')
    # Clear the ratings of the user
    new_user_ratings.clear()
    display_movies(get_movies(items_per_row*rows),'rate')

In [11]:
def close():
    root.destroy()

In [12]:
# This is called each time ANY button is clicked to submit the rated movies
def button_clicked(movie_displays=[], mode='random', term=''):
    
    # Create the dictionary of the ratings
    for item in movie_displays:
        if item.rating != 0.0:
            new_user_ratings[item.id] = float(item.rating)

    print('User has rated these movies currently:')
    print(new_user_ratings)
    print('--------------------------------------')
        
    if mode=='random':
        movie_list = get_movies(items_per_row*rows)
        display_movies(movie_list,'rate')
    elif mode=='search':
        movie_list = search_movies(term)
        if movie_list == []:
            no_movies_window()
        else:
            display_movies(movie_list,'rate')
    else: # recommend
        movie_recommendations = get_recommended_movies(new_user_ratings)
        print('Displaying recommended movies')
        display_movies(movie_recommendations,'recommend')
        

In [13]:
def no_movies_window():
    print('No movies found from search')
    info_window = tk.Toplevel(root)
    info_window.configure(bg=background_color)
    info_window.title(title)

    error_label = tk.Label(info_window, text='Η ταινία δε βρέθηκε!', wraplength=400, font=(style, 40), bg=background_color)
    error_label.grid(row=1, column=1, padx=10, pady=10)

In [14]:
# Show more info about the movie
def show_info(movie_id):
    print("Showing more info for movie: "+str(movie_id))

    link_id = movies_df[movies_df['movieId']==movie_id]['tmdbId'].iloc[0]

    response = requests.get(f"{base_url}{link_id}?api_key={api_key}")

    data     = json.loads(response.text)
    title    = data['original_title']
    overview = data['overview']
    year     = data['release_date'][:4]
    credits  = requests.get(f"{base_url}{link_id}/credits?api_key={api_key}")
    credits  = json.loads(credits.text)
    cast     = credits['cast']
    crew     = credits['crew']

    cast_list = []
    for c in cast:
        cast_list.append(c['name'])
        if len(cast_list)>4:
            break
            
    director = None
    for c in crew:
        if c['job'].lower() == 'director':
            director = c['name']
            break

    if director is None:
        print("WARNING! Director not found.")

    poster_path = data["poster_path"]
    image_path = f"{image_url}{poster_path}"
    with urllib.request.urlopen(image_path) as url:
        image_data = url.read()
    image = Image.open(io.BytesIO(image_data))
    image.thumbnail(images_dim)
    photo = ImageTk.PhotoImage(image)

    info_window = tk.Toplevel(root)
    info_window.configure(bg=background_color)
    info_window.title(title)

    image_label = tk.Label(info_window, image=photo, bg=background_color)
    image_label.image = photo  # Keep a reference to the image to prevent it from being garbage collected
    image_label.grid(row=1, column=0, padx=10, pady=10)

    overview_label = tk.Label(info_window, text=overview, wraplength=400, font=(style, title_font), bg=background_color)
    overview_label.grid(row=1, column=1, padx=10, pady=10)

    title_label = tk.Label(info_window, text=title, font=(style, title_font), bg=background_color)
    title_label.grid(row=0, column=0, columnspan=2, padx=10, pady=10)
    
    cast_names = ", ".join(cast_list)
    cast_label = tk.Label(info_window, text="Cast: " + cast_names, font=(style, title_font), bg=background_color)
    cast_label.grid(row=2, column=0, columnspan=2, padx=10, pady=10)
    
    director_label = tk.Label(info_window, text="Director: " + director, font=(style, title_font), bg=background_color)
    director_label.grid(row=3, column=0, columnspan=2, padx=10, pady=10)
    
    year_label = tk.Label(info_window, text="Year: " + year, font=(style, title_font), bg=background_color)
    year_label.grid(row=4, column=0, columnspan=2, padx=10, pady=10)
    

In [15]:
def display_start_screen():
    # Title
    title_label = tk.Label(root, text="Ταινιόπολις", font=(style, 30), bg=contrast_color)
    title_label.pack(pady=20)

    # Logo
    camera_image = Image.open("images/logo.jpg")
    camera_image.thumbnail((1000, 1000))
    camera_photo = ImageTk.PhotoImage(camera_image)
    camera_image_label = tk.Label(root, image=camera_photo, bg=background_color)
    camera_image_label.photo = camera_photo
    camera_image_label.pack(pady=20)

    start_button = tk.Button(root, text="Έναρξη!",font=(style, 16),bg=contrast_color, command=button_clicked, width=20, height=7)
    start_button.pack()

In [16]:
# Load the DataFrame from the csv
movies_df = pd.read_csv(MODEL_PATH + 'filtered_movies.csv')

movies_df["cleaned_title"] = movies_df["title"].apply(clean_title)

In [17]:
# Create a custom tkinter window and initialize it
root = tk.Tk()
root.configure(bg=background_color)
root.state(zoom)
root.title("Ταινιόπολις")

# Start screen
display_start_screen()

root.mainloop()

User has rated these movies currently:
{}
--------------------------------------
Displaying movies:
[88129, 61255, 61354, 68941, 112497, 8977, 62434, 6946, 102481, 32017, 44613, 90428]
User has rated these movies currently:
{}
--------------------------------------
Searching movies containing: inception
--------------------------------------
Displaying movies:
[79132]
User has rated these movies currently:
{79132: 5.0}
--------------------------------------
Searching movies containing: prestige
--------------------------------------
Displaying movies:
[48780]
User has rated these movies currently:
{79132: 5.0, 48780: 4.5}
--------------------------------------
Searching movies containing: interstellar
--------------------------------------
Displaying movies:
[109487]
User has rated these movies currently:
{79132: 5.0, 48780: 4.5, 109487: 4.0}
--------------------------------------
Searching movies containing: dark knight
--------------------------------------
Displaying movies:
[58559,